In [2]:
def nextBaseStep(fromIdx, toIdx, baseAggr=100):
    # get next loan idx that is closest in terms of mod baseAggr
    nextIdx = (fromIdx // baseAggr + 1) * baseAggr
    # make sure next doesn't "overstep" toIdx
    return min([nextIdx, toIdx])

def buildBaseAggrRanges(loanIdxsWhereSharesChanged, baseAggr=100):
    baseAggrRanges = loanIdxsWhereSharesChanged.copy()
    i = 0
    n = len(baseAggrRanges) - 1
    while (i < n):
        # get next loan idx in terms of mod baseAggr
        nextStep = nextBaseStep(baseAggrRanges[i], baseAggrRanges[i+1], baseAggr)
        # make sure we don't insert duplicates
        if (nextStep != baseAggrRanges[i+1]):
            baseAggrRanges.insert(i+1, nextStep)
            n = len(baseAggrRanges) - 1
        i += 1
    return baseAggrRanges

def prune(baseRanges, pruneBy=1000):
    pruned = [baseRanges[0]]
    i = 0
    n = len(baseRanges)
    while (i < n):
        # make sure we don't insert duplicates
        if pruned[-1] != baseRanges[i]:
            pruned.append(baseRanges[i])
        # check if loan idx is potentially starting point for range that can be pruned
        if baseRanges[i] % pruneBy == 0 and i + 10 < n:
            canBePruned = True
            for j in range(10):
                # check that next next 10 loan idxs can all be pruned
                if baseRanges[i+j+1] % (pruneBy/10) != 0:
                    canBePruned = False
                    break
            # if range can be pruned, add the 10th item and move forward by 10
            if canBePruned:
                pruned.append(baseRanges[i+j+1])
                i += 10
            else:
                i += 1
        else:
            i += 1
    return pruned

def packIntoClaimingTxs(pruned):
    packed = []
    i = 0
    n = len(pruned)-1
    while (i < n):
        consecutive100 = pruned[i] % 100 == 0 and pruned[i+1] % 100 == 0
        consecutive1000 = pruned[i] % 1000 == 0 and pruned[i+1] % 1000 == 0
        consecutive10000 = pruned[i] % 10000 == 0 and pruned[i+1] % 10000 == 0
        if (consecutive100 or consecutive1000 or consecutive10000):
            j = 0
            tmp = [pruned[i]]
            while (i+j+1 < n):
                diff = 100 if consecutive100 else (1000 if consecutive1000 else 10000)
                if (pruned[i+j+1] - pruned[i]) % diff == 0:
                    tmp.append(pruned[i+j+1])
                    j += 1
                else:
                    break
            i += j
            packed.append(tmp)
        else:
            packed.append([pruned[i], pruned[i+1]])
            i += 1
    return packed
            
            
# example array of where LP changed position
loanIdxsWhereSharesChanged = [4,2600]

# first construct array with "intermediate" steps according to baseAggr
baseAggrRanges = buildBaseAggrRanges(loanIdxsWhereSharesChanged, 100)
print("baseAggrRanges:\n", baseAggrRanges, "\n")

# next, try pruning by 1'000 intervals
prunedBy1000 = prune(baseAggrRanges, 1000)
print("prunedBy1000:\n", prunedBy1000, "\n")

# finally, try pruning by 10'000 intervals
prunedBy10000 = prune(prunedBy1000, 10000)
print("prunedBy10000:\n", prunedBy10000, "\n")

# get packed claiming intervals
packed = packIntoClaimingTxs(prunedBy10000)
print("packed:\n", packed, "\n")

print("loanIdxsWhereSharesChanged: ", loanIdxsWhereSharesChanged)
print("changed position {} times".format(len(loanIdxsWhereSharesChanged)))
print("need to claim {} times".format(len(packed), "\n"))

baseAggrRanges:
 [4, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600] 

prunedBy1000:
 [4, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 2100, 2200, 2300, 2400, 2500, 2600] 

prunedBy10000:
 [4, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 2100, 2200, 2300, 2400, 2500, 2600] 



<class 'MemoryError'>: 